In [2]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt # plotting
import matplotlib.image as mpimg

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os # accessing directory structure


In [3]:
DATASET_PATH = "G:/python_workspaces/fashionRecommendation/Fashion-Recommendation/myntradataset/"
print(os.listdir(DATASET_PATH))

['images', 'styles.csv']


In [4]:
df = pd.read_csv(DATASET_PATH + "styles.csv", nrows=44447, error_bad_lines=False)

C:\Users\Nadeem\AppData\Local\Temp\ipykernel_9648\2051869671.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(DATASET_PATH + "styles.csv", nrows=44447, error_bad_lines=False)
Skipping line 6044: expected 10 fields, saw 11
Skipping line 6569: expected 10 fields, saw 11
Skipping line 7399: expected 10 fields, saw 11
Skipping line 7939: expected 10 fields, saw 11
Skipping line 9026: expected 10 fields, saw 11
Skipping line 10264: expected 10 fields, saw 11
Skipping line 10427: expected 10 fields, saw 11
Skipping line 10905: expected 10 fields, saw 11
Skipping line 11373: expected 10 fields, saw 11
Skipping line 11945: expected 10 fields, saw 11
Skipping line 14112: expected 10 fields, saw 11
Skipping line 14532: expected 10 fields, saw 11
Skipping line 15076: expected 10 fields, saw 12
Skipping line 29906: expected 10 fields, saw 11
Skipping line 31625: expected 10 fields, sa

In [5]:
df.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt


In [6]:
df['gender'].value_counts()

Men       22147
Women     18631
Unisex     2161
Boys        830
Girls       655
Name: gender, dtype: int64

In [7]:
# Getting the count of each category in gender
df.groupby("gender").mean()

C:\Users\Nadeem\AppData\Local\Temp\ipykernel_9648\2629391877.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby("gender").mean()


,id,year
gender,,
Boys,32499.755422,2011.849398
Girls,30977.022901,2011.876336
Men,25194.169504,2012.565565
Unisex,21986.961129,2012.740861
Women,35772.428372,2013.175836


In [8]:
# .info() gives the information about the dataset like the number of rows and columns, the number of non-null values in each column, and the datatype of each column.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44424 entries, 0 to 44423
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  44424 non-null  int64  
 1   gender              44424 non-null  object 
 2   masterCategory      44424 non-null  object 
 3   subCategory         44424 non-null  object 
 4   articleType         44424 non-null  object 
 5   baseColour          44409 non-null  object 
 6   season              44403 non-null  object 
 7   year                44423 non-null  float64
 8   usage               44107 non-null  object 
 9   productDisplayName  44417 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 3.4+ MB


In [9]:
# .dtypes gives the datatype of each column. 
df.dtypes


id                      int64
gender                 object
masterCategory         object
subCategory            object
articleType            object
baseColour             object
season                 object
year                  float64
usage                  object
productDisplayName     object
dtype: object

In [10]:
df.describe()

,id,year
count,44424.000000,44423.000000
mean,29696.334301,2012.806497
std,17049.490518,2.126480
min,1163.000000,2007.000000
25%,14768.750000,2011.000000
50%,28618.500000,2012.000000
75%,44683.250000,2015.000000
max,60000.000000,2019.000000


In [11]:
print(f'General check for null values \n{df.isna().any()}\n')
print(f'Names of columns that contain null values: \n{df.columns[df.isna().any()].tolist()}\n')
print(f'Number of columns containing missing values: \n{len(df.columns[df.isna().any()].tolist())}\n')
print(f'Number of missing values in each column: \n{df.isna().sum()}')


General check for null values 
id                    False
gender                False
masterCategory        False
subCategory           False
articleType           False
baseColour             True
season                 True
year                   True
usage                  True
productDisplayName     True
dtype: bool

Names of columns that contain null values: 
['baseColour', 'season', 'year', 'usage', 'productDisplayName']

Number of columns containing missing values: 
5

Number of missing values in each column: 
id                      0
gender                  0
masterCategory          0
subCategory             0
articleType             0
baseColour             15
season                 21
year                    1
usage                 317
productDisplayName      7
dtype: int64


In [12]:
missingVal = df.columns[df.isna().any()]
for i in missingVal:
    print(f'{i}     {df[i].isna().sum()}')


baseColour     15
season     21
year     1
usage     317
productDisplayName     7


In [13]:
# Getting value counts of each class in the column
df['baseColour'].value_counts()


Black                9728
White                5538
Blue                 4918
Brown                3494
Grey                 2741
Red                  2455
Green                2115
Pink                 1860
Navy Blue            1789
Purple               1640
Silver               1090
Yellow                778
Beige                 749
Gold                  628
Maroon                581
Orange                530
Olive                 410
Multi                 394
Cream                 390
Steel                 315
Charcoal              228
Peach                 195
Off White             182
Skin                  179
Lavender              162
Grey Melange          146
Khaki                 139
Magenta               129
Teal                  120
Tan                   114
Mustard                97
Bronze                 95
Copper                 86
Turquoise Blue         69
Rust                   66
Burgundy               45
Metallic               43
Coffee Brown           31
Mauve       

In [14]:
df['season'].value_counts()


Summer    21472
Fall      11431
Winter     8517
Spring     2983
Name: season, dtype: int64

In [15]:
df['year'].value_counts()


2012.0    16288
2011.0    13672
2016.0     6006
2017.0     2916
2015.0     2780
2013.0     1212
2010.0      846
2018.0      405
2014.0      236
2019.0       33
2009.0       20
2008.0        7
2007.0        2
Name: year, dtype: int64

In [16]:
df['usage'].value_counts()


Casual          34406
Sports           4025
Ethnic           3208
Formal           2345
Smart Casual       67
Party              29
Travel             26
Home                1
Name: usage, dtype: int64

In [17]:
df['productDisplayName'].value_counts()

Lucera Women Silver Earrings                         82
Lucera Women Silver Pendant                          56
Lucera Women Silver Ring                             50
Catwalk Women Black Heels                            48
Q&Q Men Black Dial Watch                             42
                                                     ..
Lino Perros Women Leatherite Blue Handbag             1
Lakme True Wear Freespirit Nail Polish N525           1
Jungle Book Boys Follow The Tracks Yellow T-shirt     1
Spykar Men Navy Blue Trevor Brief                     1
Fossil Women Pink Dial Chronograph Watch ES3050       1
Name: productDisplayName, Length: 31121, dtype: int64

In [20]:
df.groupby("gender")[["baseColour"]].count()

,baseColour
gender,
Boys,830
Girls,655
Men,22136
Unisex,2161
Women,18627


In [21]:
df.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt


In [22]:
for i in df.columns:
    print(i)

id
gender
masterCategory
subCategory
articleType
baseColour
season
year
usage
productDisplayName


In [65]:
# dropping Rows having empty values 
df.dropna(inplace=True)
df

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt
...,...,...,...,...,...,...,...,...,...,...
44419,17036,Men,Footwear,Shoes,Casual Shoes,White,Summer,2013.0,Casual,Gas Men Caddy Casual Shoe
44420,6461,Men,Footwear,Flip Flops,Flip Flops,Red,Summer,2011.0,Casual,Lotto Men's Soccer Track Flip Flop
44421,18842,Men,Apparel,Topwear,Tshirts,Blue,Fall,2011.0,Casual,Puma Men Graphic Stellar Blue Tshirt
44422,46694,Women,Personal Care,Fragrance,Perfume and Body Mist,Blue,Spring,2017.0,Casual,Rasasi Women Blue Lady Perfume


In [26]:
# Showing that there are no missing values in the dataset
print("Is there any Null values ? ",df.isna().any(axis=0))


Is there any Null values ?  id                    False
gender                False
masterCategory        False
subCategory           False
articleType           False
baseColour            False
season                False
year                  False
usage                 False
productDisplayName    False
dtype: bool


In [31]:
for i in df['gender']:
    if i == 'Men' or i=='Boy'or i=='Unisex':
        print(i)

Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Unisex
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Unisex
Men
Men
Men
Men
Men
Men
Men
Unisex
Men
Unisex
Men
Men
Men
Men
Men
Men
Men
Men
Unisex
Men
Unisex
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Unisex
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Unisex
Men
Men
Unisex
Men
Men
Men
Men
Unisex
Men
Men
Men
Men
Unisex
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Unisex
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Unisex
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Unisex
Men
Unisex
Men
Men
Men
Unisex
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Unisex
Men
Men
Unisex
Men
Men
Men
Men
Men
Men
Unisex
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men
Men

In [62]:
maleDataSet = df.loc[(df['gender']=='Men') ]
maleDataSet

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt
5,1855,Men,Apparel,Topwear,Tshirts,Grey,Summer,2011.0,Casual,Inkfruit Mens Chain Reaction T-shirt
...,...,...,...,...,...,...,...,...,...,...
44414,30614,Men,Apparel,Topwear,Tshirts,Black,Summer,2012.0,Sports,Nike Men Striped Black Jersey
44415,13496,Men,Apparel,Topwear,Tshirts,Blue,Fall,2011.0,Casual,Chimp Men Teja Main Hoon Blue Tshirts
44419,17036,Men,Footwear,Shoes,Casual Shoes,White,Summer,2013.0,Casual,Gas Men Caddy Casual Shoe
44420,6461,Men,Footwear,Flip Flops,Flip Flops,Red,Summer,2011.0,Casual,Lotto Men's Soccer Track Flip Flop


In [63]:
boysDataSet = df.loc[(df['gender']=='Boys') ]
boysDataSet


,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
14,46885,Boys,Footwear,Flip Flops,Flip Flops,Navy Blue,Fall,2012.0,Casual,Ben 10 Boys Navy Blue Slippers
31,4729,Boys,Apparel,Topwear,Tshirts,Green,Summer,2011.0,Casual,Disney Kids Boy's Crew Sea Life Sialing Green ...
124,8322,Boys,Apparel,Topwear,Tshirts,Blue,Fall,2011.0,Casual,Doodle Boy's Route 66 Biker Blue Teen Kidswear
291,34036,Boys,Apparel,Topwear,Shirts,Blue,Summer,2012.0,Casual,Palm Tree Boys Check Blue Shirt
337,12994,Boys,Footwear,Shoes,Sports Shoes,Black,Fall,2011.0,Sports,ADIDAS Kids Boys Duramo Synthetic Black Sports...
...,...,...,...,...,...,...,...,...,...,...
44235,36725,Boys,Apparel,Topwear,Tshirts,Green,Summer,2012.0,Casual,Gini and Jony Boys Polo Green T-Shirt
44296,31164,Boys,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Gini and Jony Boys Blue Jeans
44361,22587,Boys,Apparel,Topwear,Tshirts,Purple,Fall,2011.0,Sports,Nike Kids Boys Purple T-shirt
44363,47110,Boys,Footwear,Shoes,Casual Shoes,Blue,Spring,2013.0,Casual,Marvel Boys Blue & Red Spiderman Shoes


In [54]:
boysDataSet.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
14,46885,Boys,Footwear,Flip Flops,Flip Flops,Navy Blue,Fall,2012.0,Casual,Ben 10 Boys Navy Blue Slippers
31,4729,Boys,Apparel,Topwear,Tshirts,Green,Summer,2011.0,Casual,Disney Kids Boy's Crew Sea Life Sialing Green ...
124,8322,Boys,Apparel,Topwear,Tshirts,Blue,Fall,2011.0,Casual,Doodle Boy's Route 66 Biker Blue Teen Kidswear
291,34036,Boys,Apparel,Topwear,Shirts,Blue,Summer,2012.0,Casual,Palm Tree Boys Check Blue Shirt
337,12994,Boys,Footwear,Shoes,Sports Shoes,Black,Fall,2011.0,Sports,ADIDAS Kids Boys Duramo Synthetic Black Sports...


In [64]:
uniSexDataSet = df.loc[(df['gender']=='Unisex') ]
uniSexDataSet

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
22,6842,Unisex,Accessories,Shoe Accessories,Shoe Accessories,Black,Summer,2011.0,Casual,Timberland Unisex Rubber Sole Brush Shoe Acces...
88,39524,Unisex,Accessories,Bags,Laptop Bag,Orange,Summer,2012.0,Casual,Peter England Unisex Orange Sleeve Bag
104,22950,Unisex,Accessories,Watches,Watches,Black,Winter,2016.0,Casual,Fastrack Unisex Black Dial Watch 9915PP13
106,8746,Unisex,Footwear,Flip Flops,Flip Flops,Navy Blue,Summer,2015.0,Casual,Crocs Kids Navy Blue Clogs
125,13841,Unisex,Accessories,Watches,Watches,White,Winter,2016.0,Casual,ADIDAS Unisex Three Small Dials White Watches
...,...,...,...,...,...,...,...,...,...,...
44318,4594,Unisex,Accessories,Bags,Rucksacks,Red,Winter,2015.0,Casual,Wildcraft Unisex Red & Grey Rucksack
44353,3745,Unisex,Apparel,Topwear,Tshirts,Yellow,Summer,2011.0,Casual,Tantra Kid's Cool Yellow Kidswear
44371,59218,Unisex,Accessories,Watches,Watches,Black,Winter,2016.0,Casual,Titan Him & Her Black Watches
44374,24670,Unisex,Footwear,Shoes,Casual Shoes,White,Fall,2010.0,Casual,Converse Unisex Floral Print Casual Shoes


In [60]:
df['gender'].value_counts()

Men       22109
Women     18357
Unisex     2126
Boys        830
Girls       655
Name: gender, dtype: int64

In [68]:
# Concatenating all datasets of Men, Boys, Unisex
frames = [maleDataSet, boysDataSet, uniSexDataSet]  
result = pd.concat(frames)
display(result)

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt
5,1855,Men,Apparel,Topwear,Tshirts,Grey,Summer,2011.0,Casual,Inkfruit Mens Chain Reaction T-shirt
...,...,...,...,...,...,...,...,...,...,...
44318,4594,Unisex,Accessories,Bags,Rucksacks,Red,Winter,2015.0,Casual,Wildcraft Unisex Red & Grey Rucksack
44353,3745,Unisex,Apparel,Topwear,Tshirts,Yellow,Summer,2011.0,Casual,Tantra Kid's Cool Yellow Kidswear
44371,59218,Unisex,Accessories,Watches,Watches,Black,Winter,2016.0,Casual,Titan Him & Her Black Watches
44374,24670,Unisex,Footwear,Shoes,Casual Shoes,White,Fall,2010.0,Casual,Converse Unisex Floral Print Casual Shoes
